In [1]:
import numpy as np
import scipy
from math import pi, sin, cos, pow, sqrt, floor, ceil
#import matplotlib
import matplotlib.pyplot as plt
import re
"""import matplotlib._color_data as mcd
xkcd = mcd.XKCD_COLORS
xkcd = matplotlib._color_data.XKCD_COLORS
#print(list(xkcd.values())[2])
colors_list=list(xkcd.values())
import random
random.shuffle(colors_list)"""

'import matplotlib._color_data as mcd\nxkcd = mcd.XKCD_COLORS\nxkcd = matplotlib._color_data.XKCD_COLORS\n#print(list(xkcd.values())[2])\ncolors_list=list(xkcd.values())\nimport random\nrandom.shuffle(colors_list)'

In [2]:
from scipy.linalg import eigh_tridiagonal

### Pot (potential) class usage
#### Constructing
Prefered constructors :<br>
   >  -pot.wVsin(phi,r,dx,n,V1n,V2n) for sinusoidal V1<br>
    -pot.wVharm(phi,r,dx,n,V1n,V2n) for harmonic V1

(If default constructor is used instead, further methods need to be called before plotting)

#### Plots
Plotting methods include : 
   > -plotVA() for the energy <br>
    > -plotIPR() for the IPR<br>
    > -plotV("sin"/"harm") to plot the shape of V1 <br>

Plotting optional arguments include:<br>
   > -"rangex_a_b" : plot for x between a and b<br>
    > -"rangey_a_b" : plot for y between a and b<br>
    > -"same" : 
    > -"parab" : plot the theorical parabola associated with free particle hamiltonian<br>
    > -"cos" : plot the energy cosine associated with the dx lattice

#### Other
Use eigen_check() to check for the egeinvector normalization

See below for examples

In [2]:
class pot:
    def __init__(self,phi,r,dx,n,V1n,V2n,*args):   #default constructor
        self.phi=phi                           #phase of the second lattice
        self.r=r                               #period of the second lattice
        self.dx=dx                             #discretisation intervall length
        self.n=n                               #number of points
        self.nm=n
        self.minn=0
        self.q = 2
        for a in args:
            if re.match("nm(.)+",a):
                self.nm=int(re.findall(r"nm_(.+)", a)[0])
            if re.match("minn(.)+",a):
                self.minn=int(re.findall(r"minn_(.+)", a)[0])
            if re.match('q(.)+',a):
                self.q = int(re.findall(r'q_(.+)', a)[0])
        self.L=ceil(self.n*self.dx)
        self.V1n=V1n                           #amplitude of the first potential                      
        self.V2n=V2n                           #amplitude of the second potential 
        self.H = None
        self.IPRq = {}
    
    @classmethod
    def wVsin(cls,phi,r,dx,n,V1n,V2n,*args):   #full constructor for lattice case (sine potential)
        if "L" in args:
            n=floor(n/dx)                    #"L" means that the argument passed is the length L of the system, L=(n-1)dx
        p=pot(phi,r,dx,n,V1n,V2n,*args)
        if "nobound" in args:
            p.H_Vsinnb()
            #p.eigen_tri()
        elif "bound" in args:
            p.H_Vsinb()
        elif "lnrhighbound" in args:
            p.H_Vsinlnrhb()
        elif "nrhighbound" in args:
            print("nrighbound really ")
            p.H_Vsinnrhb()
        elif "highbound" in args:
            p.H_Vsinhb()
            p.eigen()
        elif "rhighbound" in args:
            p.H_Vsinrhb()
            p.eigen()
        else : 
            p.H_Vsin()
            p.eigen()
        p.InversePartRatio() #<----------- IPR_q calculation
        #p.NPartRatio()
        #p.eigen_comp()
        return(p)
    
    @classmethod
    def wVharm(cls,phi,r,dx,n,V1n,V2n,*args):   #full constructor for harmonic trap
        if "L" in args:
            n=floor(n/dx)
        p=pot(phi,r,dx,n,V1n,V2n)
        p.H_Vharm()
        p.eigen()
        p.InversePartRatio()
        p.eigen_comp()
        return(p)
    
    @classmethod
    def wTransferV1(cls, ClsP, vc1): #ONLY WORKS FOR sinlnrhb
        #self.V1n*sin(pi*(i+self.n/2)*dx)**2+self.V2n*sin(pi*(i+self.n/2)*dx*r+self.phi)**2-2*value
        self = ClsP.copy()
        value=-1/((pi**2)*(self.dx**2))
        DeltaV = vc1 - ClsP.V1n
        self.V1n = vc1
        self.H = self.H + np.array([DeltaV*sin(pi*(i+self.n/2)*self.dx)**2 for i in range(self.n)])
        (self.Va, self.Ve) = scipy.linalg.eigh_tridiagonal(self.H,[value for _ in range(self.n-1)],select='i', select_range=[0,self.nm-1])
        self.Ve=self.Ve/sqrt(sum(np.power(self.Ve[:,1],2)*self.dx))
        self.InversePartRatio()
        self.NPartRatio()
        return self
    
    def copy(self):
        new = pot(self.phi, self.r, self.dx, self.n, self.V1n, self.V2n)
        new.H = self.H
        new.Va = self.Va
        new.Ve = self.Ve
        new.nm = self.nm
        new.IPR = self.IPR
        return new
    
    def H_Vsin(self):                          #Hamiltonian calculation
        n,dx,r=self.n,self.dx,self.r
        value=-1/((pi**2)*(dx**2))
        self.H=np.eye(n)
        for i in range(n):
            self.H[i][(i-1)%n]= value
            self.H[i][i]= self.V1n*sin(pi*(i+n/2)*dx)**2+self.V2n*sin(pi*(i+n/2)*dx*r+self.phi)**2-2*value
            self.H[i][(i+1)%n]= value 
            
    def H_Vharm(self):
        n,dx,r=self.n,self.dx,self.r
        value=-1/((pi**2)*(dx**2))
        self.H=np.eye(n)
        for i in range(n):
            self.H[i][(i-1)%n]= value
            self.H[i][i]= self.V1n*(pi*(i*dx-(dx*(n-1)/2)))**2+self.V2n*sin(pi*(i)*dx*r+self.phi)**2-2*value
            self.H[i][(i+1)%n]= value 
    
    def H_Vsinnb(self):   
        n,dx,r=self.n,self.dx,self.r
        value=-1/((pi**2)*(dx**2))
        H=[self.V1n*sin(pi*(i+self.n/2)*dx)**2+self.V2n*sin(pi*(i+self.n/2)*dx*r+self.phi)**2-2*value for i in range(n)]
        if self.n!=self.nm:
            (self.Va,self.Ve)=scipy.linalg.eigh_tridiagonal(H,[value for _ in range(n-1)],select='i', select_range=[self.minn,self.minn+self.nm-1])
        else:
            if (self.n>5000): print("Warning, very high number of modes requested")
            (self.Va,self.Ve)=scipy.linalg.eigh_tridiagonal(H,[value for _ in range(n-1)])
        self.Ve=self.Ve/sqrt(sum(np.power(self.Ve[:,1],2)*self.dx))
    
    def H_Vsinb(self):   
        n,dx,r=self.n,self.dx,self.r
        value=-1/((pi**2)*(dx**2))        
        self.H=np.eye(n)
        for i in range(n):
            self.H[i][(i-1)%n]= value
            self.H[i][i]= self.V1n*sin(pi*(i+n/2)*dx)**2+self.V2n*sin(pi*(i+n/2)*dx*r+self.phi)**2-2*value
            self.H[i][(i+1)%n]= value 
        self.H=scipy.sparse.csr_matrix(self.H)
        if self.n!=self.nm:
            (self.Va,self.Ve)=scipy.sparse.linalg.eigsh(self.H,self.nm)
        else:
            if (self.n>5000): print("Warning, very high number of modes requested")
            (self.Va,self.Ve)=scipy.sparse.linalg.eigsh(H)
        self.Ve=np.array([v/sqrt(sum(np.power(v,2)*dx)) for v in self.Ve])
        
        
    """def H_Vsinnb(self):                          #Hamiltonian calculation
        n,dx,r=self.n,self.dx,self.r
        value=-1/((pi**2)*(dx**2))
        self.H=np.eye(n)
        for i in range(n):
            if i:self.H[i][(i-1)%n]= value
            self.H[i][i]= self.V1n*sin(pi*(i+n/2)*dx)**2+self.V2n*sin(pi*(i+n/2)*dx*r+self.phi)**2-2*value
            if i<n-1:self.H[i][(i+1)%n]= value """
            
    def H_Vsinhb(self):                          #Hamiltonian calculation
        n,dx,r=self.n,self.dx,self.r
        value=-1/((pi**2)*(dx**2))
        self.H=np.eye(n)
        for i in range(1,n-1):
            self.H[i][(i-1)%n]= value
            self.H[i][i]= self.V1n*sin(pi*(i+n/2)*dx)**2+self.V2n*sin(pi*(i+n/2)*dx*r+self.phi)**2-2*value
            self.H[i][(i+1)%n]= value 
        for i in (0,n-1):
            self.H[i][(i-1)%n]= value
            self.H[i][i]= 9999999999999999999999999
            self.H[i][(i+1)%n]= value 
      
    def H_Vsinnrhb(self):                          #Hamiltonian calculation
        n,dx,r=self.n,self.dx,self.r
        value=-1/((pi**2)*(dx**2))
        H=[self.V1n*sin(pi*(i+self.n/2)*dx)**2+self.V2n*sin(pi*(i+self.n/2)*dx*r+self.phi)**2-2*value for i in range(n)]
        H[0]=9999
        H[n-1]=9999
        if self.n!=self.nm:
            (self.Va,self.Ve)=scipy.linalg.eigh_tridiagonal(H,[value for _ in range(n-1)],select='i', select_range=[0,self.nm-1])
        else:
            if (self.n>5000): print("Warning, very high number of modes requested")
            (self.Va,self.Ve)=scipy.linalg.eigh_tridiagonal(H,[value for _ in range(n-1)])
        self.Ve=self.Ve/sqrt(sum(np.power(self.Ve[:,1],2)*self.dx))
        
    def H_Vsinlnrhb(self):                          #Hamiltonian calculation
        n,dx,r=self.n,self.dx,self.r
        value=-1/((pi**2)*(dx**2))
        H=[self.V1n*sin(pi*(i+self.n/2)*dx)**2+self.V2n*sin(pi*(i+self.n/2)*dx*r+self.phi)**2-2*value for i in range(n)]
        for i in range(0,20):H[i]=9999
        for i in range(n-20,n):H[i]=9999
        self.H = H
        if self.n!=self.nm:
            (self.Va,self.Ve)=scipy.linalg.eigh_tridiagonal(H,[value for _ in range(n-1)],select='i', 
                                                            select_range=[0,self.nm-1])
        else:
            if (self.n>5000): print("Warning, very high number of modes requested")
            (self.Va,self.Ve)=scipy.linalg.eigh_tridiagonal(H,[value for _ in range(n-1)])
        self.Ve=self.Ve/sqrt(sum(np.power(self.Ve[:,1],2)*self.dx))
    
    def H_Vsinrhb(self):                          #Hamiltonian calculation
        n,dx,r=self.n,self.dx,self.r
        value=-1/((pi**2)*(dx**2))
        self.H=np.eye(n)
        for i in range(1,n-1):
            self.H[i][(i-1)%n]= value
            self.H[i][i]= self.V1n*sin(pi*(i+n/2)*dx)**2+self.V2n*sin(pi*(i+n/2)*dx*r+self.phi)**2-2*value
            self.H[i][(i+1)%n]= value 
        for i in (0,n-1):
            if i:self.H[i][(i-1)%n]= value
            self.H[i][i]= 9999
            if i<n-1:self.H[i][(i+1)%n]= value 
            
    def eigen(self):                         #eigen value/vectors calculation
        (self.Va,self.Ve)=np.linalg.eigh(self.H)
        
        self.Ve=self.Ve/sqrt(sum(np.power(self.Ve[:,0],2)*self.dx))  #eigenvectors are normalized accordingto the norm of the second vector
    
    def eigen_tri(self):                         #eigen value/vectors calculation
        if p.n!=p.nm:
            p.eigen()
        else:
            (self.Va,self.Ve)=np.linalg.eigh(self.H)
        self.Ve=self.Ve/sqrt(sum(np.power(self.Ve[:,1],2)*self.dx))  #eigenvectors are normalized accordingto the norm of the second vector
  
    def eigen_check(self):                   #normalization calculation
        print(self.Ve)     
        plt.plot(range(self.n),sum(np.power(self.Ve,2)*self.dx))
        plt.ylim(0,3)
        plt.show()

    def InversePartRatio2(self):             #old IPR function
        self.IPR= np.zeros(int((self.n+1)/2)) 
        self.IPR[0]= sum(np.power(self.Ve[:,0],4))/(sum(np.power(self.Ve[:,0],2))**2*self.dx)
        for i in range(1,int((self.n+1)/2)):
            self.IPR[i]= sum(np.power((np.power(self.Ve[:,2*i-1],2))+(np.power(self.Ve[:,2*i],2)),2))/(sum(np.power(self.Ve[:,2*i-1],2)+np.power(self.Ve[:,i*2],2))**2*self.dx)
    
    def InversePartRatio(self):             #<----------- IPR_q calculation
        '''
            Computes the IPR_q of the problem, where q is given by the class paramter.
            If q is set to be 0 it is understood as meaning that we do not want to
            compute the IPR_q and therefore we skip it (for computational time).
        '''
        if self.q == 0: #If q is 0 then skip
            return
        self.IPR= np.zeros(self.nm) 
        for i in range(0,self.nm): #For each eigenstate compute the IPR_q
            self.IPR[i]= sum((np.power(self.Ve[:,i],2*self.q))/
                             (sum(np.power(self.Ve[:,i],2))**self.q*self.dx))

    def NPartRatio(self):             #new IPR function
        self.NPR= np.zeros(self.nm) 
        for i in range(0,self.nm):
            self.NPR[i]= sum((np.power(self.Ve[:,i],4)))

    def eigen_comp(self):
        self.Va_nond=np.zeros(int((self.n+1)/2))
        self.Va_nond[0]=self.Va[0]
        for i in range(1,int((self.n+1)/2)):
            self.Va_nond[i]=self.Va[2*i-1]    
  

    #Plotting functions below
     
    def preplot(self,*args):              #Called before plotting, set color
        cl,psize="",0.7
        for a in args:
            if re.match("cl_(.)+",a):
                cl=re.findall(r"cl_(.+)", a)[0]
            if re.match("sz_(.)+",a):
                psize=float(re.findall(r"sz_(.+)", a)[0])
        return (cl,psize)
            
    
    def plot(self,*args):                #Base plotting function, parse all the optional arguments
        for a in args:
            if re.match("rangex(.)+",a):
                xi,xf=(float(e) for e in re.findall(r"rangex_((?:[0-9]|\.)+)_(?P<en>(?:\d|\.)+)", a)[0])
                plt.xlim((xi,xf))
            if re.match("rangey(.)+",a):
                yi,yf=(float(e) for e in re.findall(r"rangey_((?:[0-9]|\.)+)_(?P<en>(?:\d|\.)+)", a)[0])
                plt.ylim((yi,yf))
            if re.match("leg",a):
                plt.legend()
             
            
            if re.match("slinex(.)+",a):
                x,yi,yf=(float(e) for e in re.findall(r"slinex_((?:[0-9]|\.)+)_((?:[0-9]|\.)+)_((?:[0-9]|\.)+)", a)[0])
                plt.plot([x, x], [yi, yf], 'k-', lw=2)
            if re.match("sliney(.)+",a):
                y,xi,xf=(float(e) for e in re.findall(r"sliney_((?:[0-9]|\.)+)_((?:[0-9]|\.)+)_((?:[0-9]|\.)+)", a)[0])
                plt.plot([xi, xf], [y, y], 'k-', lw=2)            
            if re.match("linex(.)+",a):
                x,yi,yf=(float(e) for e in re.findall(r"slinex_((?:[0-9]|\.)+)_((?:[0-9]|\.)+)_((?:[0-9]|\.)+)", a)[0])
                plt.plot([x, x], [yi, yf], 'k-', lw=2)
            if re.match("liney(.)+",a):
                y,xi,xf=(float(e) for e in re.findall(r"sliney_((?:[0-9]|\.)+)_((?:[0-9]|\.)+)_((?:[0-9]|\.)+)", a)[0])
                plt.plot([xi, xf], [y, y], 'k-', lw=2)
        if not "same" in args: plt.show()
    
    def plotV(self,*args):              #potential plotting function
        cl,psize=self.preplot(*args)
        if "actual" in args: 
            plt.plot(range(self.nm), [self.H[i][i]-2/((pi**2)*(dx**2)) for i in range(self.n)],
                     "."+cl,label='V1sin',markersize=0.7)
        if "V1sin" in args:
            plt.plot(range(self.nm), self.V1n*np.sin(pi* (np.linspace(0,self.n-1,self.n)+self.n/2)*self.dx) **2,
                     "."+cl,label='V1sin',markersize=0.7)
        if "V1harm" in args:
            plt.plot(range(self.nm), self.V1n*(pi* (self.dx*np.linspace(0,self.n-1,self.n)-(self.dx*(self.n-1)/2)) )**2,
                     "."+cl,label='V1harm',markersize=0.7)    
        if "all" in args:
            plt.plot(np.linspace(0,self.dx*(self.n-1),self.n),
                     self.V1n*np.sin(pi*(np.linspace(0,self.n-1,self.n)+self.n/2)*self.dx)**2+
                     self.V2n*np.sin(pi*(np.linspace(0,self.n-1,self.n)+self.n/2)*self.dx*self.r+self.phi)**2,
                     "."+cl,label='V1harm',markersize=0.7) 
        self.plot(*args)
            
    def plotVA(self,*args): 
        cl,psize=self.preplot(*args)
        if ("p" in args): 
            plt.plot(np.linspace(0,pi/self.dx,self.nm), self.Va, "."+cl,label='calculee (cos)',markersize=psize)
            plt.xlabel( 'p/hbar(1/a1)')
        elif (not "test" in args) : 
            plt.plot(range(self.nm), self.Va, "."+cl,label='calculee (cos)',markersize=psize)
            plt.xlabel( 'numero du mode')
        plt.title('representation de l energie des modes')
        plt.ylabel(r'energy ($E_R$)')  
        if "cos" in args:
            plt.plot(range(self.nm), [4*pow(int((i)/2),2)/(((self.n-1)*self.dx)**2) for i in range(self.nm)],
                     ".",label='thcos',markersize=0.7)
        if "cos_old" in args:
            ampli_E=self.Va[self.n-1]/2
            print("ampli_E",ampli_E)
            plt.plot(range(self.nm), ampli_E*np.cos((np.linspace(0,self.nm-1,self.nm)-self.n)*pi/self.n)+ampli_E,
                     ".",label='cos',markersize=0.7)
        if "parab" in args:
            parab=[4*pow(int((i+1)/2),2)/((self.L)**2) for i in range(self.nm)]
            print("parab :",parab[self.n-1])
            plt.plot(range(self.n), parab, ".",label='theorique (parabole)',markersize=psize)
        if "V1harm" in args:
            vlin=np.ones(self.nm)
            for i in range(self.nm):
                vlin[i]= (2*i+1)*sqrt(self.V1n) 
            plt.plot(range(self.n), vlin, ".",label='theorique (harmonique)',markersize=psize)
        self.plot(*args)  
        #plt.show()
       
    def plotIPR(self,*args):
        cl,psize=self.preplot("sz_2",*args)
        if ("mode" in args): 
            plt.plot(range(self.nm), self.IPR, "."+cl,label='', markersize=psize)
            plt.xlabel( 'numero du mode')
            plt.ylabel( 'IPR')
        elif ("rev" in args): 
            plt.plot(self.Va, self.IPR, "."+cl,label='', markersize=psize)
            plt.xlabel( 'energie')
            plt.ylabel( 'IPR')
        else : 
            #plt.plot(self.IPR, self.Va_nond, "b.",label='', markersize=2)
            plt.plot(self.IPR, self.Va, "."+cl,label='', markersize=psize)
            plt.xlabel( 'IPR')
            plt.ylabel( 'energie')
        plt.title('representation de l\' IPR')
        #plt.xlim((-0.02,0.02))
        self.plot(*args)
        #plt.show()
      
    def plotVects(self,l,*args):
        #TODO
        absi=np.ones(self.n)
        for i in range(self.n):
            absi[i]=self.dx*i
        if ("p1" in args): 
            for i in l:
                plt.plot(absi,self.Ve[:,i],label=str(i))
        if ("s1" in args): 
            for i in l:
                plt.plot(absi,[sin(i/self.n*j*pi) for j in range(self.n)],label="sin "+str(i))
        if ("p2" in args): 
            for i in l:
                plt.plot(absi,np.power(self.Ve[:,i],2),label=str(i)+'au carre')
                #plt.plot(absi,np.power(self.Ve[:,i],2), 'r.', markersize=0.5,label='i au carre')
        plt.legend()
        plt.xlabel('x')
        plt.ylabel('eigenvector amplitude')
        plt.title('eigenvector')
        self.plot(*args)

    


    